# EECS16A: Homework 7A

# Question 1: Labeling Patients

In [1]:
%pylab inline
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import sys

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Part B
A = np.load('gene_data_train.npy')
b = np.load('diabetes_train.npy')


In [3]:
#Calculate the coefficients using the train set
trans = A.T
sqr = trans.dot(A)
pseudoinv = (np.linalg.inv(sqr)).dot(trans)
x = pseudoinv.dot(b)
print(x)

[[ 0.12131475]
 [-0.15253102]
 [-0.36111722]
 [-0.06427341]
 [ 0.95936096]]


In [4]:
#Test if these coefficients give the correct answer for the train set
b_new = np.sign(A.dot(x))
print(b - b_new)

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]


In [5]:
# Part C
A_test = np.load('gene_data_test.npy')
b_test = np.load('diabetes_test.npy')


In [6]:
#Use your alpha coefficients on the test set and check their acuracy
b_calc = np.sign(A_test.dot(x))
print(b_calc)
print(b_calc - b_test)

[[1.]
 [1.]
 [1.]
 [1.]]
[[0.]
 [0.]
 [2.]
 [0.]]


# Question 2: Noise Cancelling Headphones

In [23]:
%matplotlib inline

import numpy as np
from matplotlib.pyplot import plot
from scipy.io import wavfile

from audio_support import wavPlayer
from audio_support import loadSounds
from audio_support import recordAmbientNoise

### Part c)
In the following cell, implement the least squares solution to 
$$\min\limits_{\vec x} \left\| A \vec x - \vec b \right\|$$

In [24]:
def doLeastSquares(A,b):
    # BEGIN
    ATA = A.T.dot(A);
    ATA_inv = np.linalg.inv(ATA);
    ATA_inv_AT = ATA_inv.dot(A.T);
    x = ATA_inv_AT.dot(b);  
    # END
    return x;  

### Part d)
Use your least squares solution to find the gamma that minimizes the effect of noise given:

$$\vec n = \begin{bmatrix} 0.10\\ 0.37\\-0.45\\0.068\\0.036 \end{bmatrix};
	\vec r_A = \begin{bmatrix} 0\\ 0.11\\-0.31\\-0.012\\-0.018 \end{bmatrix};
	\vec r_B = \begin{bmatrix} 0\\ 0.22\\-0.20\\0.080\\0.056 \end{bmatrix};
	\vec r_C = \begin{bmatrix} 0\\ 0.37\\-0.44\\0.065\\0.038 \end{bmatrix}$$


In [25]:
n1 = 0.10;
n2 = 0.37;
n3 = -0.45;
n4 = 0.068;
n5 = 0.036;

a1 = 0;
a2 = 0.11;
a3 = -0.31;
a4 = -0.012;
a5 = -0.018;

b1 = 0;
b2 = 0.22;
b3 = -0.20;
b4 = 0.080;
b5 = 0.056;

c1 = 0;
c2 = 0.37;
c3 = -0.44;
c4 = 0.065;
c5 = 0.038;

# BEGIN

R = np.array([[a1,b1,c1],
     [a2,b2,c2],
     [a3,b3,c3],
     [a4,b4,c4],
     [a5,b5,c5]]);
n = np.array([n1,n2,n3,n4,n5]);

gamma = doLeastSquares(R,-n);

# END
print(gamma)

[-0.0882878  -0.09300071 -0.91842393]


<font color="red">Report the results for your gamma-vector.

### Part e)
First, we'll load the sounds from the included .wav files.

In [26]:
[music_Fs, music_y, noise1_y, noise1_Fs, noise2_y, noise2_Fs] = loadSounds();

In [27]:
noise1_y

array([ 0.10751015,  0.36674906, -0.45176844, ...,  0.06786902,
        0.03579603,  0.07659678])

We can use the following function to listen to our signals throughout this notebook. 

<font color="red">Listen to each of the loaded sounds (`music_y`, `noise1_y`, and `noise2_y`). What do you hear?

In [28]:
wavPlayer(music_y, music_Fs)

Add the first noise to the signal and listen to the result.

In [29]:
noisyMusic = music_y + noise1_y;
wavPlayer(noisyMusic, music_Fs)

<font color="red">Add the second noise to the signal and listen to the result.

In [30]:
# BEGIN
noisyMusic = music_y + noise2_y;
wavPlayer(noisyMusic, music_Fs)
# END

### Part f)
Next, we will simulate the recording of `noise1` using a simulated microphone array.

In [31]:
numberOfMicrophones = 3;
R = recordAmbientNoise(noise1_y,noise1_Fs,numberOfMicrophones);


<font color="red">In the cell below, calculate the gamma-vector using the least squares approach (you should calculate `gamma` from `R` and `noise1_y`). 

In [32]:
# BEGIN
gamma = doLeastSquares(R,-noise1_y);
# END

<font color="red">In the cell below, create the noise cancellation signal by multiplying `R` and `gamma`. Add the result to `music_y` (with the right sign) to get `signalFromSpeaker`.

In [33]:
# BEGIN
noiseCancellation = R.dot(gamma);
signalFromSpeaker = music_y+noiseCancellation;
# END

### Part g)
Generate the signal at the listener's ear by adding the speaker signal (`signalFromSpeaker`) to the original noise signal (`noise1_y`).

In [34]:
# BEGIN
signalAtEar = signalFromSpeaker + noise1_y;
# END

Listen to the noisy and noise-cancelled signal.

In [35]:
wavPlayer(noisyMusic, music_Fs)
wavPlayer(signalAtEar, music_Fs)

<font color="red">What difference can you hear between these signals?

### Part h)
Now, we'll see how well this gamma works for other noise. 

We will run through the simulation again, but this time, we will just use the gamma from before instead of going through a training step.

In [36]:
noisyMusic_2 = music_y + noise2_y;
R_2 = recordAmbientNoise(noise2_y,noise2_Fs,numberOfMicrophones);
# BEGIN
noiseCancellation_2 = R_2.dot(gamma);
signalFromSpeaker_2 = music_y+noiseCancellation_2;
signalAtEar_2 = signalFromSpeaker_2 + noise2_y;
# END 

wavPlayer(noisyMusic_2, music_Fs)
wavPlayer(signalAtEar_2, music_Fs)

<font color="red"> What do you hear in the noise-cancelled signal?